<a href="https://colab.research.google.com/github/amoghjalan/Optical-Character-Recognition/blob/main/OCR_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>